In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/amir/PycharmProjects/rqc_internship/sensor_repo


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import join as jn
import os
import yaml
import torch
from torchinfo import summary
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset


In [50]:
from video_module import *

In [7]:
with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)
with open('pathes.yaml') as conf_file:
    path_config = yaml.safe_load(conf_file)

pressure_path = path_config['p_video_path']
signal_path = path_config['s_video_path']


In [19]:
v_ds = Video_dataset(pressure_path, signal_path)
v_ds.split_to_chains(30)

In [21]:
for x in v_ds:
    print(x[0].shape, x[1].shape)

(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)


In [27]:
v_dl = DataLoader(v_ds, batch_size=5)
signal, pressure = next(iter(v_dl))
signal.shape

torch.Size([5, 30, 4, 64])

In [31]:
subDL = SubDataLoader(signal, initial_pressure=pressure[:, 0])


In [33]:
for i in range(1, len(subDL)):
    input = subDL.get_next_batch()
    subDL.set_previous_pressure(pressure[:, i])
    if i == 7:
        break

In [37]:
input[0].shape, input[1].shape, input[2].shape

(torch.Size([5, 64, 64]), torch.Size([5, 4, 64]), torch.Size([5, 4, 64]))

In [38]:
from models_src import *

In [39]:
model = TestModel((64, 64), (4, 64))
model(*input).shape

torch.Size([5, 64, 64])

In [47]:
signal.shape

torch.Size([5, 30, 4, 64])

In [46]:
predict_chain_batch(model, signal, initial_pressure=pressure[:, 0]).shape

torch.Size([5, 256, 64])

In [66]:
from video_module import *

In [60]:
optim = torch.optim.Adam(model.parameters(), lr=1e-4)
fit_epoch(model, v_ds, torch.nn.MSELoss(), optim, 30, 5, 'cpu')

torch.Size([5, 30, 4, 64])
30
0.23783138394355774
torch.Size([5, 30, 4, 64])
30
0.25069481134414673
torch.Size([5, 30, 4, 64])
30
0.23790596425533295
torch.Size([5, 30, 4, 64])
30
0.2516731917858124
torch.Size([5, 30, 4, 64])
30
0.23971502482891083
torch.Size([5, 30, 4, 64])
30
0.23799049854278564
torch.Size([5, 30, 4, 64])
30
2677.748046875
torch.Size([5, 30, 4, 64])
30
6886.7255859375
torch.Size([2, 30, 4, 64])
30
7402.98291015625


1491.3241199702024

In [49]:
torch.rand(10, 10)[:, None].shape

torch.Size([10, 1, 10])

In [58]:
signal.size(0)

5

In [63]:
signal[None].shape

torch.Size([1, 5, 30, 4, 64])

In [64]:
signal_np = np.load(jn(signal_path, '0.npy'))

In [68]:
signal_np.shape

(297, 4, 64)

In [79]:
predict(model, signal_np, 'cpu').shape

(296, 64, 64)

In [73]:
torch.tensor(signal_np)[None].shape

torch.Size([1, 297, 4, 64])